## Introduction

In this lab, we will manually reconstruct an HDR image from an SDR source and a Gain Map.
We will:
1. Load the SDR HEIC image.
2. Load the Gain Map PNG.
3. Apply the Apple Gain Map formula.
4. Save the result as a 16-bit PNG with PQ encoding and correct metadata.

In [ ]:
import colour
import io
import numpy as np
import os
import png
import struct
import zlib
import xml.etree.ElementTree as ET

from PIL import Image
from pillow_heif import register_heif_opener

# Set up data directory - works in mybinder and locally
DATA_DIR = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'data')
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    print(f"Created data directory: {DATA_DIR}")
else:
    print(f"Data directory found: {DATA_DIR}")

# List files in data directory
if os.path.exists(DATA_DIR):
    files = os.listdir(DATA_DIR)
    if files:
        print(f"Files in data directory: {files}")
    else:
        print("Data directory is empty. Please upload your HEIC and gain map PNG files.")

## PNG Binary Helpers

Standard Python libraries often struggle to save 16-bit PNGs with specific HDR metadata (the cICP chunk). We will write a helper to manually inject this chunk, which tells the operating system (Windows/macOS) that the image uses the BT.2020 color primaries and the PQ transfer function.

In [ ]:
# Helper Functions (PNG & Metadata)
def create_png_chunk(chunk_type: bytes, data: bytes) -> bytes:
    """Creates a valid PNG chunk: [Length][Type][Data][CRC]"""
    chunk = struct.pack(">I", len(data)) + chunk_type + data
    crc = zlib.crc32(chunk_type + data) & 0xffffffff
    chunk += struct.pack(">I", crc)
    return chunk

def save_pq_png(filename, width, height, data_16bit):
    """Saves a 16-bit PNG with a cICP chunk for PQ HDR."""
    writer = png.Writer(width=width, height=height, bitdepth=16, greyscale=False)
    buffer = io.BytesIO()
    writer.write(buffer, data_16bit)
    png_bytes = buffer.getvalue()
    
    # cICP Chunk: Primaries=9 (BT.2020), Transfer=16 (PQ), Matrix=0, Range=1
    cicp_chunk = create_png_chunk(b'cICP', b'\x09\x10\x00\x01')
    
    with open(filename, 'wb') as f:
        f.write(png_bytes[:33])    # Signature and IHDR
        f.write(cicp_chunk)        # cICP Metadata
        f.write(png_bytes[33:])    # IDAT and IEND
    print(f"✓ Saved: {filename}")

print("PNG helper functions loaded successfully.")

## Color Science & Metadata Logic

Here we handle the core logic. We need to:

- **Parse XMP**: Apple stores the "Headroom" (how much brighter the HDR is compared to SDR) in XML metadata inside the image.
- **Adapt Color Space**: Move from the phone's display space (P3) to the standard HDR space (BT.2020).

In [ ]:
# Color Science Logic
def adaptation_to_bt2020(image_array, input_colorspace):
    """Convert from input colorspace to BT.2020 using Bradford chromatic adaptation."""
    return colour.RGB_to_RGB(
        image_array,
        input_colourspace=input_colorspace,
        output_colourspace=colour.models.RGB_COLOURSPACE_BT2020,
        chromatic_adaptation_transform="Bradford",
        apply_cctf_decoding=False,
        apply_cctf_encoding=False,
    )

def parse_headroom_from_xmp(pil_image):
    """Extracts the HDRGainMapHeadroom value from XMP metadata."""
    xmp_data = pil_image.info.get("xmp") or pil_image.info.get("XML:com.adobe.xmp")
    if not xmp_data:
        print("Warning: No XMP metadata found. Using default headroom of 1.0")
        return 1.0 # Default

    try:
        namespaces = {
            "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
            "HDRGainMap": "http://ns.apple.com/HDRGainMap/1.0/"
        }
        root = ET.fromstring(xmp_data)
        target_tag = root.find(".//HDRGainMap:HDRGainMapHeadroom", namespaces)
        if target_tag is not None and target_tag.text:
            return float(target_tag.text)
    except Exception as e:
        print(f"Warning: Error parsing XMP metadata: {e}")
        pass
    return 1.0

print("Color science functions loaded successfully.")

## The Reconstruction Pipeline

This is the heart of the script. We implement the Apple Gain Map formula:

$$L_{hdr} = L_{sdr} \times (1 + (Headroom - 1) \times GainMap)$$

This formula allows us to recover the HDR brightness values from the standard dynamic range image and the gain map.

In [ ]:
def apply_apple_gain_map(input_path, gain_map_path, ref_white=203, peak_nits=1000):
    """
    Apply Apple's gain map formula to reconstruct HDR from SDR + gain map.
    
    Args:
        input_path: Path to SDR HEIC image
        gain_map_path: Path to gain map PNG
        ref_white: Reference white level in nits (default: 203)
        peak_nits: Peak brightness in nits (default: 1000)
    
    Returns:
        HDR image in PQ encoding
    """
    sdr_colorspace = colour.models.RGB_COLOURSPACE_DISPLAY_P3
    register_heif_opener()
    
    print(f"Loading SDR image from: {input_path}")
    # 1. Load Images
    sdr_img = Image.open(input_path)
    sdr = np.array(sdr_img).astype(np.float32) / 255.0
    print(f"  SDR image size: {sdr_img.size}, shape: {sdr.shape}")
    
    print(f"Loading gain map from: {gain_map_path}")
    gain_map = Image.open(gain_map_path)
    gain_map_resized = gain_map.resize(sdr_img.size, resample=Image.BICUBIC)
    gain = np.array(gain_map_resized).astype(np.float32) / 255.0
    print(f"  Gain map size: {gain_map.size}, resized shape: {gain.shape}")
    
    # Expand dimensions if needed
    if gain.ndim == 2: 
        gain = np.stack([gain]*3, axis=-1)
        print("  Expanded grayscale gain map to 3 channels")
    elif gain.shape[2] == 1: 
        gain = np.concatenate([gain]*3, axis=-1)
        print("  Expanded single-channel gain map to 3 channels")

    # 2. Linearize
    print("Linearizing SDR and gain map...")
    sdr_linear = sdr_colorspace.cctf_decoding(sdr)
    gain_linear = colour.models.oetf_inverse_BT709(gain)
    
    # 3. Apply Gain
    headroom = parse_headroom_from_xmp(gain_map)
    print(f"Headroom detected: {headroom}")
    hdr_linear_p3 = sdr_linear * (1.0 + (headroom - 1.0) * gain_linear)
    print(f"Applied gain map formula")
    
    # 4. Convert to BT.2020 & PQ
    print("Converting to BT.2020 color space...")
    hdr_linear_2020 = adaptation_to_bt2020(hdr_linear_p3, sdr_colorspace)
    hdr_abs_nits = hdr_linear_2020 * ref_white
    print(f"Applying PQ encoding (ref_white={ref_white}, peak_nits={peak_nits})...")
    hdr_pq = colour.models.eotf_inverse_BT2100_PQ(np.clip(hdr_abs_nits, 0, peak_nits))
    
    print("HDR reconstruction complete!")
    return hdr_pq

print("Gain map application function loaded successfully.")

## Execute the Pipeline

Now we'll run the complete pipeline. Make sure you have:
1. `112_115.HEIC` - The source SDR image
2. `112_115-urn_com_apple_photo_2020_aux_hdrgainmap.png` - The gain map

Both files should be in the `data/` folder.

In [ ]:
# Define file paths
input_image_path = os.path.join(DATA_DIR, "112_115.HEIC")
gain_map_path = os.path.join(DATA_DIR, "112_115-urn_com_apple_photo_2020_aux_hdrgainmap.png")
output_filename = os.path.join(DATA_DIR, "output_112_115_PQ.png")

print("=" * 60)
print("HDR GAIN MAP RECONSTRUCTION PIPELINE")
print("=" * 60)

# Check if files exist
if not os.path.exists(input_image_path):
    print(f"❌ ERROR: SDR image not found at: {input_image_path}")
    print("   Please upload '112_115.HEIC' to the data/ folder.")
elif not os.path.exists(gain_map_path):
    print(f"❌ ERROR: Gain map not found at: {gain_map_path}")
    print("   Please upload '112_115-urn_com_apple_photo_2020_aux_hdrgainmap.png' to the data/ folder.")
else:
    print("✓ Input files found!\n")
    
    try:
        # Run the pipeline
        hdr_pq = apply_apple_gain_map(input_image_path, gain_map_path, ref_white=203, peak_nits=1000)
        
        # Convert to 16-bit
        print("\nConverting to 16-bit format...")
        hdr_uint16 = (np.clip(hdr_pq, 0, 1) * 65535).astype(np.uint16)
        h, w, c = hdr_uint16.shape
        flat_rows = hdr_uint16.reshape(h, w * c)
        
        # Save the output
        print(f"\nSaving HDR output to: {output_filename}")
        save_pq_png(output_filename, w, h, flat_rows)
        
        print("\n" + "=" * 60)
        print("✓ SUCCESS! HDR reconstruction complete.")
        print(f"✓ Output saved to: {output_filename}")
        print("=" * 60)
        
        # Display some statistics
        print(f"\nOutput Statistics:")
        print(f"  Resolution: {w}x{h}")
        print(f"  Bit depth: 16-bit")
        print(f"  Color space: BT.2020")
        print(f"  Transfer function: PQ (Perceptual Quantizer)")
        print(f"  File size: {os.path.getsize(output_filename) / (1024*1024):.2f} MB")
        
    except Exception as e:
        print(f"\n❌ ERROR during processing: {e}")
        import traceback
        traceback.print_exc()

## Download the Output

If you're running this on mybinder.org, you can download the output file by:
1. Looking in the file browser on the left
2. Navigating to the `data/` folder
3. Right-clicking on `output_112_115_PQ.png` and selecting "Download"

Alternatively, run the cell below to create a download link:

In [ ]:
from IPython.display import FileLink, display

if os.path.exists(output_filename):
    print("Click the link below to download your HDR image:")
    display(FileLink(output_filename))
else:
    print("Output file not found. Please run the pipeline first.")